# Recursive Ekphrasis Gym

## Processing `(noun,verb)` pairs

In [ ]:
! wget https://github.com/kbooten/ekphrasisgym/raw/main/noun2verb_tuples_with_count.json

--2022-08-12 10:55:07--  https://github.com/kbooten/ekphrasisgym/raw/main/noun2verb_tuples_with_count.json
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/kbooten/ekphrasisgym/main/noun2verb_tuples_with_count.json [following]
--2022-08-12 10:55:07--  https://raw.githubusercontent.com/kbooten/ekphrasisgym/main/noun2verb_tuples_with_count.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 49097393 (47M) [text/plain]
Saving to: ‘noun2verb_tuples_with_count.json.3’

noun2verb_tuples_wi 100%[===================>]  46.82M   274MB/s    in 0.2s    

2022-08-12 10:55:07 (274 MB/s) - ‘noun2verb_tuples_w

In [ ]:
import json

with open('noun2verb_tuples_with_count.json','r') as f:
  noun2verb_tuples_with_count = json.load(f)

In [ ]:
noun2verb_tuples_with_count[:4]

[[['grace', 'kill', 1], 2],
 [['child', 'sum', 1], 4],
 [['world', 'see', 1], 715],
 [['thou', 'find', 1], 132]]

In [ ]:
[n for n,c in noun2verb_tuples_with_count if n[0]=="mouse"]#.count(1)

[['mouse', 'have', 1],
 ['mouse', 'eat', 1],
 ['mouse', 'find', 1],
 ['mouse', 'begin', 1],
 ['mouse', 'marry', 1],
 ['mouse', 'repay', 1],
 ['mouse', 'see', 1],
 ['mouse', 'build', 1],
 ['mouse', 'worry', 1],
 ['mouse', 'poke', 1],
 ['mouse', 'transform', 1],
 ['mouse', 'fret', 1],
 ['mouse', 'take', 1],
 ['mouse', 'deserve', 1],
 ['mouse', 'say', 1],
 ['mouse', 'make', 1],
 ['mouse', 'help', 1],
 ['mouse', 'show', 1],
 ['mouse', 'think', 1],
 ['mouse', 'leave', 1],
 ['mouse', 'bring', 1],
 ['mouse', 'enter', 1],
 ['mouse', 'feast', 1],
 ['mouse', 'prefer', 1],
 ['mouse', 'recollect', 1],
 ['mouse', 'publish', 1],
 ['mouse', 'answer', 1],
 ['mouse', 'consume', 1],
 ['mouse', 'lead', 1],
 ['mouse', 'gnaw', 1],
 ['mouse', 'visit', 1],
 ['mouse', 'give', 1],
 ['mouse', 'run', 1],
 ['mouse', 'nibble', 1],
 ['mouse', 'notice', 1],
 ['mouse', 'recommend', 1],
 ['mouse', 'hear', 1],
 ['mouse', 'gird', 1],
 ['mouse', 'stir', 1],
 ['mouse', 'cross', 1],
 ['mouse', 'represent', 1],
 ['mouse', '

Using WordNet to make sure a token is a word.

In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.corpus import wordnet as wn

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
wn.synsets("wiggle",pos=wn.VERB)

[Synset('jiggle.v.01')]

In [ ]:
from collections import defaultdict

noun2verbs = defaultdict(list)

In [ ]:
min_count = 3

for tup,count in noun2verb_tuples_with_count:
  noun,verb,num = tup
  if count>=min_count: ### don't add very rare words
    if (noun.isalpha() and verb.isalpha() and noun.islower() and verb.islower()):
      if wn.synsets(verb,pos=wn.VERB)!=[]:
        noun_plus_num = noun+"~"+str(num)
        noun2verbs[noun_plus_num]+=[verb]*count

Get rid of values that are the same as the key.

In [ ]:
for key,values in noun2verbs.items():
  noun2verbs[key] = [v for v in values if v.split("~")[0]!=key]

In [ ]:
noun2verbs["pig~1"][:3]

['root', 'root', 'root']

Sometimes a value is just a letter.  Filter out really short words.

In [ ]:
for key,values in noun2verbs.items():
  noun2verbs[key] = [v for v in values if len(v)>2]

### Rank by TF-IDF

In [ ]:
sets_of_words = list(noun2verbs.values())

In [ ]:
total_number_of_sets = len(sets_of_words)
total_number_of_sets

14116

In [ ]:
from collections import defaultdict

word2doc_count = defaultdict(int)

In [ ]:
for s in sets_of_words:
  s = list(set(s))
  for t in s:
    word2doc_count[t]+=1

In [ ]:
word2doc_count['eat']

461

In [ ]:
noun2verbs_and_weights = {}

In [ ]:
for key,words in noun2verbs.items():
  if len(words)!=0: ## no empty sets
    unique_words = list(set(words))
    verbs_and_weights = []
    for w in unique_words:
      tf = words.count(w)/len(words)
      idf = total_number_of_sets/word2doc_count[w]
      tfidf = tf * idf
      verbs_and_weights.append((w,tfidf))
    verbs_and_weights.sort(key=lambda x:x[1], reverse=True)
    noun2verbs_and_weights[key]=verbs_and_weights[:20]

Make dictionary more concise.

```
dict[key] = {
  "num":1, ## or 2
  "verbs":[("animadvert",11.45),...]
}
```

First, only keep singular *or* plural of noun, whichever has more results (more likely to be correct).

In [ ]:
all_keys_without_number = list(set([n.split("~")[0] for n in noun2verbs_and_weights]))

In [ ]:
for k in all_keys_without_number:
  if ((k+"~1" in noun2verbs_and_weights) and (k+"~2" in noun2verbs_and_weights)):
    if len(noun2verbs_and_weights[k+"~1"])>len(noun2verbs_and_weights[k+"~2"]):
      del noun2verbs_and_weights[k+"~2"]
    else:
      del noun2verbs_and_weights[k+"~1"]

In [ ]:
noun2verbs_and_weights_reformatted = {}

for key,values in noun2verbs_and_weights.items():
  # print(key)
  # print(key.split("~"))
  key,num = key.split("~")
  noun2verbs_and_weights_reformatted[key]={}
  noun2verbs_and_weights_reformatted[key]["num"]=num
  noun2verbs_and_weights_reformatted[key]["verbs"]=values

In [ ]:
noun2verbs_and_weights_reformatted["laugh"]

{'num': '1',
 'verbs': [('subside', 5.744048830111902),
  ('jar', 4.418499100086079),
  ('reassure', 4.335131192537284),
  ('laugh', 4.3253379744818545),
  ('greet', 3.2549610037300782),
  ('enliven', 2.2673876960968036),
  ('interrupt', 1.6633346033142575),
  ('exasperate', 1.5955691194755284),
  ('anger', 1.346261444557477),
  ('disconcert', 1.346261444557477),
  ('startle', 1.326750409129108),
  ('dispel', 1.2859810813683363),
  ('brighten', 1.1262840843356672),
  ('seal', 1.1046247750215197),
  ('applaud', 1.0770091556459815),
  ('awake', 1.0349637531733156),
  ('answer', 1.0136556759021005),
  ('chill', 0.9735675983240512),
  ('drown', 0.8975076297049848),
  ('sound', 0.7853191759918616)]}

In [ ]:
with open('noun2verbs_and_weights.json','w') as f:
  json.dump(noun2verbs_and_weights_reformatted,f)

In [ ]:
from google.colab import files
files.download('/content/noun2verbs_and_weights.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

***